<a href="https://colab.research.google.com/github/ncn-foreigners/job-ads-classifier/blob/main/job_classification_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Update python version

In [22]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [25]:
!git clone https://github.com/ncn-foreigners/job-ads-classifier.git 

Cloning into 'job-ads-classifier'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 95 (delta 28), reused 42 (delta 4), pack-reused 0
Unpacking objects: 100% (95/95), done.
tcmalloc: large alloc 1471086592 bytes == 0x558d7e95e000 @  0x7f5aefb982a4 0x558d4159511f 0x558d4157225b 0x558d41526f33 0x558d414cb22a 0x558d414cb6e6 0x558d414e8451 0x558d414e89e9 0x558d414e8f13 0x558d4158de12 0x558d4142f162 0x558d41415a65 0x558d41416725 0x558d4141572a 0x7f5aeeedfc87 0x558d4141577a
tcmalloc: large alloc 2206621696 bytes == 0x558dd644e000 @  0x7f5aefb982a4 0x558d4159511f 0x558d4157225b 0x558d41526f33 0x558d414cb22a 0x558d414cb6e6 0x558d414e8451 0x558d414e89e9 0x558d414e8f13 0x558d4158de12 0x558d4142f162 0x558d41415a65 0x558d41416725 0x558d4141572a 0x7f5aeeedfc87 0x558d4141577a
Filtering content: 100% (14/14), 2.09 GiB | 46.58 MiB/s, done.


In [2]:
!python --version

Python 3.7.15


In [3]:
!python -m pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [4]:
!pip install virtualenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 38.8 MB/s eta 0:00:00


In [5]:
!virtualenv classifier

created virtual environment CPython3.7.15.final.0-64 in 1008ms
  creator CPython3Posix(dest=/content/classifier, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==22.2.2, setuptools==65.3.0, wheel==0.37.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [6]:
!source /content/classifier/bin/activate

In [ ]:
!pip list

## Clone repository

## Colab settings


In [8]:
from google.colab.data_table import DataTable
DataTable.max_columns = 60

## Set up environment

Install required packages

In [24]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!pip install -r ./req-colab.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 8.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.5/541.5 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 46.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.7/361.7 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.1/708.1 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.4/660.4 kB 35.3 MB/s eta 0:00:00
  

## Working with trained models

In [26]:
models_dir="/content/job-ads-classifier/models"

Load required modules

In [27]:
import sys
sys.path.insert(1, '/content/job-ads-classifier')

In [28]:
# Numpy, pandas i matplotlib jako pomocnicze biblioteki
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from job_offers_classifier.load_save import load_to_df, save_obj, load_obj
from job_offers_classifier.job_offers_utils import create_hierarchy, fix_class_str, remove_classes, filter_classes, remap_classes
from job_offers_classifier.job_offers_classfier import *
from sklearn.model_selection import train_test_split
import torch
from torchmetrics import MetricCollection, Recall, Accuracy, ConfusionMatrix

Example data -- job offers from Public Sercice

In [5]:
kprm_df=pd.read_xml("https://nabory.kprm.gov.pl/pls/serwis/app.xml", xpath = "./new/oferta")
kprm_df = kprm_df.replace("\\<\\!\\[CDATA\\[ ", "", regex=True)
kprm_df = kprm_df.replace(" \\]\\]\\>", "", regex=True)
kprm_df = kprm_df.replace(r'<[^<>]*>', '', regex=True)
kprm_df = kprm_df.replace('&nbsp;', ' ', regex=True)
kprm_df = kprm_df.replace('&dash;', ' ', regex=True)
kprm_df = kprm_df.replace('&oacute;', ' ', regex=True)
kprm_df = kprm_df.replace('\r', ' ', regex=True)
kprm_df[["unid", "nazwa_firmy", "stanowisko", "do_spraw", "poledodatkowelista1", "poledodatkowelista3"]].head(n=2)

,unid,nazwa_firmy,stanowisko,do_spraw,poledodatkowelista1,poledodatkowelista3
0,108581,Ministerstwo Spraw Zagranicznych w Warszawie,inspektor,ekspedycji informacji niejawnych,Prowadzi dzienniki ewidencji materiałów niejaw...,Warunki dotyczące charakteru pracy na stanowis...
1,108580,Powiatowy Inspektorat Nadzoru Budowlanego w Op...,inspektor nadzoru budowlanego,kontroli,Przygotowywanie i opracowywanie projektów decy...,Praca w siedzibie Inspektoratu przy wykorzysta...


In [6]:
kprm_df.poledodatkowelista1[1]

'Przygotowywanie i opracowywanie projektów decyzji, postanowień i innych pism z zakresu właściwości Powiatowego Inspektora Nadzoru Budowlanego.Prowadzenie postępowań administracyjnych.Prowadzenie kontroli budów, robót budowlanych i utrzymania obiektów budowlanych.Przyjmowanie zawiadomień o zamierzonym terminie rozpoczęcia robót budowlanych, o zakończeniu budowy oraz wniosków o udzielenie pozwolenia na użytkowanie.Prowadzenie ewidencji i przygotowywanie wymaganych sprawozdań.'

### Logistic regression model using tf-df matrix

In [30]:
model_paper_linear = LinearJobOffersClassifier()
model_paper_linear.load(f"{models_dir}/linear_model")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.2 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.23.2 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [48]:
kprm_df_zaw1 = model_paper_linear.predict(
    X_text=list(kprm_df["poledodatkowelista1"]),
    output_level=0,
    format="dataframe"
)

Processing X ...
Processing text ...


100%|██████████| 450/450 [00:02<00:00, 198.20it/s]


Transforming text to tf-idf ...
Predicting ...


100%|██████████| 450/450 [00:00<00:00, 1316.23it/s]


In [49]:
kprm_df_zaw1.head(n=5)

,0,1,2,3,4,5,6,7,8,9
0,0.000250,0.000640,0.652122,0.280615,0.008959,0.001145,0.001836,0.023618,0.005647,2.512277e-02
1,0.000002,0.000026,0.993806,0.006005,0.000007,0.000002,0.000001,0.000099,0.000003,7.051406e-07
2,0.000128,0.029682,0.892026,0.067866,0.005381,0.000537,0.000158,0.002359,0.000752,1.063783e-03
3,0.000114,0.001121,0.372066,0.611876,0.001536,0.004502,0.001836,0.005977,0.000155,7.720769e-04
4,0.000153,0.006518,0.583565,0.387423,0.003716,0.009227,0.002546,0.003270,0.003195,3.423351e-04


In [50]:
kprm_df_zaw6_top5 = model_paper_linear.predict(
    X_text=list(kprm_df["poledodatkowelista1"]),
    output_level="last",
    format="dataframe",
    top_k=5
)

Processing X ...
Processing text ...


100%|██████████| 450/450 [00:02<00:00, 212.90it/s]


Transforming text to tf-idf ...
Predicting ...


100%|██████████| 450/450 [00:00<00:00, 1497.50it/s]


In [53]:
kprm_df_zaw6_top5.head(n=5)

,class_1,class_2,class_3,class_4,class_5,prob_1,prob_2,prob_3,prob_4,prob_5
0,242110,334102,334101,334306,243390,0.218574,0.155126,0.039867,0.029361,0.028181
1,242211,311211,214202,311204,311212,0.991834,0.003184,0.001015,0.000571,0.000427
2,242217,241203,242211,242223,241290,0.071343,0.064142,0.044098,0.037160,0.036222
3,325512,225290,225211,225209,314204,0.367114,0.063721,0.049297,0.027435,0.023879
4,325512,225211,225290,225207,225101,0.350336,0.331611,0.058897,0.019308,0.018973


### Transformer model

In [54]:
model_paper_trans = TransformerJobOffersClassifier(gpus=1, threads=5)
model_paper_trans.load(f"{models_dir}/transformer_model")

Initializing TransformerClassifier with model_name=allegro/herbert-base-cased, output_type=linear, num_labels=2905, learning_rate=1e-05, weight_decay=0.01, warmup_steps=50 ...


Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.sso.sso_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [55]:
pracuj_kod2 = model_paper_trans.predict(
    X=list(kprm_df["poledodatkowelista1"]),
    output_level=0,
    format="dataframe"
)

Initializing TransformerDataModule with model_name=allegro/herbert-base-cased, max_seq_length=512, train/eval_batch_size=8/8, num_workers=5 ...
Setting up TransformerDataModule ...


TypeError: ignored